In [1]:
import numpy as np
import tensorflow as tf
from bidict import bidict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
ENCODER = bidict({
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, 
    '6': 6, '7': 7, '8': 8, '9': 9,
    'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15,
    'G': 16, 'H': 17, 'I': 18, 'J': 19, 'K': 20, 'L': 21,
    'M': 22, 'N': 23, 'O': 24, 'P': 25, 'Q': 26, 'R': 27,
    'S': 28, 'T': 29, 'U': 30, 'V': 31, 'W': 32, 'X': 33,
    'Y': 34, 'Z': 35
})

In [4]:
data = np.loadtxt('../data/mnist_train.csv', delimiter=',')
labels = data[:, 0]
images = data[:, 1:]

In [5]:
images = images / 255.0

In [6]:
images = images.reshape(-1, 28, 28, 1)

In [7]:
labels_encoded = to_categorical(labels)
labels_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

In [9]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

Epoch 1/10
1500/1500 [==============================] - 41s 25ms/step - loss: 0.1471 - accuracy: 0.9543 - val_loss: 0.0613 - val_accuracy: 0.9799
Epoch 2/10
1500/1500 [==============================] - 39s 26ms/step - loss: 0.0466 - accuracy: 0.9852 - val_loss: 0.0416 - val_accuracy: 0.9868
Epoch 3/10
1500/1500 [==============================] - 43s 29ms/step - loss: 0.0310 - accuracy: 0.9903 - val_loss: 0.0420 - val_accuracy: 0.9870
Epoch 4/10
1500/1500 [==============================] - 55s 37ms/step - loss: 0.0219 - accuracy: 0.9929 - val_loss: 0.0382 - val_accuracy: 0.9892
Epoch 5/10
1500/1500 [==============================] - 52s 35ms/step - loss: 0.0173 - accuracy: 0.9943 - val_loss: 0.0366 - val_accuracy: 0.9902
Epoch 6/10
1500/1500 [==============================] - 60s 40ms/step - loss: 0.0124 - accuracy: 0.9962 - val_loss: 0.0420 - val_accuracy: 0.9899
Epoch 7/10
1500/1500 [==============================] - 82s 55ms/step - loss: 0.0102 - accuracy: 0.9967 - val_loss: 0.0421 -

In [12]:
model.save('../chiffre.model')

INFO:tensorflow:Assets written to: ../chiffre.model\assets


INFO:tensorflow:Assets written to: ../chiffre.model\assets
